In [ ]:
#### this script is for task 2 ####

import os,sys
import subprocess
import threading

working_path="./simhome/"

SIM_BIN="./bin/sim-outorder"
CONFIG="-config ./base2.txt"


def run_a_cmd(cmd, is_print=False):
  # print(cmd, flush=True)
  cmd = "cd " + working_path + ";" + cmd
  result = subprocess.run(cmd,shell=True)
  if is_print==True:
    print("done cmd", flush= True)


In [ ]:
from tqdm import tqdm
# clean all old results
# run_a_cmd("rm -rf ./configs/task_2_3")

# run pipeline with in-order issue
# -issue:inorder                true

# instruction commit B/W (insts/cycle)
# -commit:width                     1 


# instruction issue B/W (insts/cycle)
# -issue:width                      1 

# instruction decode B/W (insts/cycle)
# -decode:width                     1 

ISSUE_WIDTH_list =[
  1,
  2,
  # 3,
  4,
  # 5,
  # 6,
  # 7,
  8
]

DECODE_WIDTH_list =[
  1,
  2,
  # 3,
  4,
  # 5,
  # 6,
  # 7,
  8
]

COMMIT_WIDTH_list =[
  1,
  2,
  # 3,
  4,
  # 5,
  # 6,
  # 7,
  8
]

# instruction fetch queue size (in insts)
# -fetch:ifqsize                    4 
IF_Queue_size_list=[
  # 2,
  # 4,
  8
  ]
# register update unit (RUU) size
# -ruu:size                        16
RUU_size_list = [
  # 16,
  # 32,
  # 64,
  128
  ]
# load/store queue (LSQ) size
# -lsq:size                         8
# LSQ_list = RUU_size_list/2
# total number of integer ALU's available
# -res:ialu                         1 
ALUs_list = [
  # 1,
  # 2,
  # 3,
  # 4,
  # 5,
  # 6,
  # 7,
  8
  ]
# total number of integer multiplier/dividers available
# -res:imult                        1 
Mults_list = [
              # 1,
              # 2,
              # 3,
              4
              ]
# issue instructions down wrong execution paths
# -issue:wrongpath               false
Issue_wrong_path_list = [
  # "True", 
  "False"
  ]



app_map = {
  # $DIR/Stats_dijkstra.txt ${SHOME}/apps/dijkstra/dijkstra_small  ${SHOME}/apps/dijkstra/input.dat
  "dijkstra_small": "./apps/dijkstra/dijkstra_small ./apps/dijkstra/input.dat > /dev/null",
  # ${DIR}/Stats_stringsearch-cabce.txt ${SHOME}/apps/stringsearch/search_small  &
  "stringsearch-cabce": "./apps/stringsearch/search_small > /dev/null",
  # ${SHOME}/apps/qsort/qsort_small  ${SHOME}/apps/qsort/input_small.dat
  "qsort_small": "./apps/qsort/qsort_small ./apps/qsort/input_small.dat > /dev/null",
  # ${SHOME}/apps/gsm-untoast/untoast -fps -c ${SHOME}/apps/gsm-untoast/small.au.run.gsm
  "gsm-untoast": "./apps/gsm-untoast/untoast -fps -c ./apps/gsm-untoast/small.au.run.gsm > /dev/null",
  # ${SHOME}/apps/jpeg-cjpeg/cjpeg -dct int -progressive -opt -outfile ${SHOME}/configs/${DIR}/jpeg-cjpeg_output_small_encode.jpeg ${SHOME}/apps/jpeg-cjpeg/input_small.ppm
  "jpeg-cjpeg": "./apps/jpeg-cjpeg/cjpeg -dct int -progressive -opt -outfile /tmp/jpeg-cjpeg_output_small_encode.jpeg ./apps/jpeg-cjpeg/input_small.ppm > /dev/null"
}


In [ ]:

print(app_map.keys())
plist = []
for issue in ISSUE_WIDTH_list:
  for ifqs in IF_Queue_size_list:
    for ruus in RUU_size_list:
      lsq = ruus//2
      for alus in ALUs_list:
        for mults in Mults_list:
          for iwp in Issue_wrong_path_list:
            output_folder = "iss_%d_ifqs_%d_ruus_%d_lsq_%d_alus_%d_mults_%d_iwp_%s" % (issue,ifqs,ruus,lsq,alus,mults,iwp)    
            output_folder = os.path.join("./configs/task_2_3/", output_folder)
            run_a_cmd("mkdir -p " + output_folder)
            for app in app_map.keys(): 
              stats_output_path=os.path.join(output_folder,"Stats_" + app + ".txt" )
              cmd= SIM_BIN + " "  + CONFIG + " "\
                  + "-bpred comb  -issue:inorder false " \
                  + "-issue:width %d " % issue \
                  + "-fetch:ifqsize %d " % ifqs \
                  + "-ruu:size %d " % ruus \
                  + "-lsq:size %d " % lsq \
                  + "-res:ialu %d " % alus \
                  + "-res:imult %d " % mults \
                  + "-issue:wrongpath %s " % iwp \
                  + " -redir:sim " + stats_output_path \
                  + " "  + app_map[app]
              t = threading.Thread(target=run_a_cmd,args=(cmd,))
              # run_a_cmd(cmd, True)
              t.start()
              plist.append(t)
              if len(plist)>= os.cpu_count() * 2 :
                print("%d thread have been strated!, waitting them to be done!"% len(plist))
                # wait all thread done
                for i in tqdm(range(len(plist)), "batch processing..."):
                  t= plist[i]
                  t.join()
                  # print("%d:%d done!"%(i+1, len(plist)))
                plist=[]

for i in tqdm(range(len(plist)), "batch processing..."):
  t= plist[i]
  t.join()

In [ ]:
# DECODE WIDTH
print(app_map.keys())
plist = []
for dec in DECODE_WIDTH_list:
  for ifqs in IF_Queue_size_list:
    for ruus in RUU_size_list:
      lsq = ruus//2
      for alus in ALUs_list:
        for mults in Mults_list:
          for iwp in Issue_wrong_path_list:
            output_folder = "dec_%d_ifqs_%d_ruus_%d_lsq_%d_alus_%d_mults_%d_iwp_%s" % (dec,ifqs,ruus,lsq,alus,mults,iwp)    
            output_folder = os.path.join("./configs/task_2_3/", output_folder)
            run_a_cmd("mkdir -p " + output_folder)
            for app in app_map.keys(): 
              stats_output_path=os.path.join(output_folder,"Stats_" + app + ".txt" )
              cmd= SIM_BIN + " "  + CONFIG + " "\
                  + "-bpred comb  -issue:inorder false " \
                  + "-issue:width 1 " \
                  + "-decode:width %d " % dec \
                  + "-fetch:ifqsize %d " % ifqs \
                  + "-ruu:size %d " % ruus \
                  + "-lsq:size %d " % lsq \
                  + "-res:ialu %d " % alus \
                  + "-res:imult %d " % mults \
                  + "-issue:wrongpath %s " % iwp \
                  + " -redir:sim " + stats_output_path \
                  + " "  + app_map[app]
              t = threading.Thread(target=run_a_cmd,args=(cmd,))
              # run_a_cmd(cmd, True)
              t.start()
              plist.append(t)
              if len(plist)>= os.cpu_count() * 2 :
                print("%d thread have been strated!, waitting them to be done!"% len(plist))
                # wait all thread done
                for i in tqdm(range(len(plist)), "batch processing..."):
                  t= plist[i]
                  t.join()
                  # print("%d:%d done!"%(i+1, len(plist)))
                plist=[]

for i in tqdm(range(len(plist)), "batch processing..."):
  t= plist[i]
  t.join()

In [ ]:
# COMMIT WIDTH
print(app_map.keys())
plist = []
for comm in COMMIT_WIDTH_list:
  for ifqs in IF_Queue_size_list:
    for ruus in RUU_size_list:
      lsq = ruus//2
      for alus in ALUs_list:
        for mults in Mults_list:
          for iwp in Issue_wrong_path_list:
            output_folder = "comm_%d_ifqs_%d_ruus_%d_lsq_%d_alus_%d_mults_%d_iwp_%s" % (comm,ifqs,ruus,lsq,alus,mults,iwp)    
            output_folder = os.path.join("./configs/task_2_3/", output_folder)
            run_a_cmd("mkdir -p " + output_folder)
            for app in app_map.keys(): 
              stats_output_path=os.path.join(output_folder,"Stats_" + app + ".txt" )
              cmd= SIM_BIN + " "  + CONFIG + " "\
                  + "-bpred comb  -issue:inorder false " \
                  + "-issue:width 1 " \
                  + "-decode:width 1 " \
                  + "-commit:width %d " % comm \
                  + "-fetch:ifqsize %d " % ifqs \
                  + "-ruu:size %d " % ruus \
                  + "-lsq:size %d " % lsq \
                  + "-res:ialu %d " % alus \
                  + "-res:imult %d " % mults \
                  + "-issue:wrongpath %s " % iwp \
                  + " -redir:sim " + stats_output_path \
                  + " "  + app_map[app]
              t = threading.Thread(target=run_a_cmd,args=(cmd,))
              # run_a_cmd(cmd, True)
              t.start()
              plist.append(t)
              if len(plist)>= os.cpu_count() * 2 :
                print("%d thread have been strated!, waitting them to be done!"% len(plist))
                # wait all thread done
                for i in tqdm(range(len(plist)), "batch processing..."):
                  t= plist[i]
                  t.join()
                  # print("%d:%d done!"%(i+1, len(plist)))
                plist=[]

for i in tqdm(range(len(plist)), "batch processing..."):
  t= plist[i]
  t.join()

In [ ]:
# COMMIT WIDTH
print(app_map.keys())
plist = []
for comm in COMMIT_WIDTH_list:
  for ifqs in IF_Queue_size_list:
    for ruus in RUU_size_list:
      lsq = ruus//2
      for alus in ALUs_list:
        for mults in Mults_list:
          for iwp in Issue_wrong_path_list:
            output_folder = "iss_2_dec_2_comm_%d_ifqs_%d_ruus_%d_lsq_%d_alus_%d_mults_%d_iwp_%s" % (comm,ifqs,ruus,lsq,alus,mults,iwp)    
            output_folder = os.path.join("./configs/task_2_3/", output_folder)
            run_a_cmd("mkdir -p " + output_folder)
            for app in app_map.keys(): 
              stats_output_path=os.path.join(output_folder,"Stats_" + app + ".txt" )
              cmd= SIM_BIN + " "  + CONFIG + " "\
                  + "-bpred comb  -issue:inorder false " \
                  + "-issue:width 2 " \
                  + "-decode:width 2 " \
                  + "-commit:width %d " % comm \
                  + "-fetch:ifqsize %d " % ifqs \
                  + "-ruu:size %d " % ruus \
                  + "-lsq:size %d " % lsq \
                  + "-res:ialu %d " % alus \
                  + "-res:imult %d " % mults \
                  + "-issue:wrongpath %s " % iwp \
                  + " -redir:sim " + stats_output_path \
                  + " "  + app_map[app]
              t = threading.Thread(target=run_a_cmd,args=(cmd,))
              # run_a_cmd(cmd, True)
              t.start()
              plist.append(t)
              if len(plist)>= os.cpu_count() * 2 :
                print("%d thread have been strated!, waitting them to be done!"% len(plist))
                # wait all thread done
                for i in tqdm(range(len(plist)), "batch processing..."):
                  t= plist[i]
                  t.join()
                  # print("%d:%d done!"%(i+1, len(plist)))
                plist=[]

for i in tqdm(range(len(plist)), "batch processing..."):
  t= plist[i]
  t.join()

In [ ]:
# COMMIT WIDTH
print(app_map.keys())
plist = []
for comm in COMMIT_WIDTH_list:
  for ifqs in IF_Queue_size_list:
    for ruus in RUU_size_list:
      lsq = ruus//2
      for alus in ALUs_list:
        for mults in Mults_list:
          for iwp in Issue_wrong_path_list:
            output_folder = "iss_8_dec_8_comm_%d_ifqs_%d_ruus_%d_lsq_%d_alus_%d_mults_%d_iwp_%s" % (comm,ifqs,ruus,lsq,alus,mults,iwp)    
            output_folder = os.path.join("./configs/task_2_3/", output_folder)
            run_a_cmd("mkdir -p " + output_folder)
            for app in app_map.keys(): 
              stats_output_path=os.path.join(output_folder,"Stats_" + app + ".txt" )
              cmd= SIM_BIN + " "  + CONFIG + " "\
                  + "-bpred comb  -issue:inorder false " \
                  + "-issue:width 8 " \
                  + "-decode:width 8 " \
                  + "-commit:width %d " % comm \
                  + "-fetch:ifqsize %d " % ifqs \
                  + "-ruu:size %d " % ruus \
                  + "-lsq:size %d " % lsq \
                  + "-res:ialu %d " % alus \
                  + "-res:imult %d " % mults \
                  + "-issue:wrongpath %s " % iwp \
                  + " -redir:sim " + stats_output_path \
                  + " "  + app_map[app]
              t = threading.Thread(target=run_a_cmd,args=(cmd,))
              # run_a_cmd(cmd, True)
              t.start()
              plist.append(t)
              if len(plist)>= os.cpu_count() * 2 :
                print("%d thread have been strated!, waitting them to be done!"% len(plist))
                # wait all thread done
                for i in tqdm(range(len(plist)), "batch processing..."):
                  t= plist[i]
                  t.join()
                  # print("%d:%d done!"%(i+1, len(plist)))
                plist=[]

for i in tqdm(range(len(plist)), "batch processing..."):
  t= plist[i]
  t.join()